# Personal Search

## Context search

This code aims to perform a search in texts loaded in the vector-store. The result seeks to be more of a quote than an elaborate answer. It will be used later to answer questions.

## Busca de contexto

Esse código tem como objetivo realizar uma busca em textos carregados no vector-store. O resultado busca ser mais uma citação do que uma resposta elaborada. Será usada mais a frente para responder perguntas.

## Import libraries

In [1]:
from IPython.display import display, Markdown
from langchain import hub
from search.context import utils

## Configuration

In [2]:
# Model from Ollama
model = "llama3:8b"

## Retriever Instance

In [3]:
retriever, db = utils.retriver_context()

## Prompt

In [4]:
# Validated Prompt 
prompt = hub.pull(owner_repo_commit="personal-search/context-search")

## Question

In [5]:
question = "O que falta aos pregadores de hoje?"

## Search

In [6]:
final_answer = utils.search_context(model=model, temperature=0, retriever=retriever, prompt=prompt, question=question)

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama

# Multi Query: Different Perspectives
template = """Você é um assistente de modelo de linguagem de IA. Sua tarefa é gerar cinco
diferentes versões da pergunta do usuário para recuperar documentos relevantes de um vetor
base de dados. Ao gerar múltiplas perspectivas sobre a pergunta do usuário, seu objetivo é ajudar
o usuário supera algumas das limitações da busca por similaridade baseada em distância.
Forneça essas perguntas alternativas separadas por novas linhas. Pergunta original: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)



# LLM
llm = Ollama(model="llama3:8b")

generate_queries = (
    prompt_perspectives 
    | llm.bind(stop=['<|eot_id|>'])
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [12]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})

In [14]:
from operator import itemgetter
#from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

# RAG
template = """Responda a seguinte questão baseada no contexto:

{context}

Questão: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

#llm = ChatOpenAI(temperature=0)
llm = Ollama(model="llama3:8b")

final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm.bind(stop=['<|eot_id|>'])
    | StrOutputParser()
)

final_rag_answer = final_rag_chain.invoke({"question":question})

A questão é inspirada pelo livro "Somos Todos Teólogos" do reverendo R.C. Sproul, e mais especificamente pela página 240 da obra.

Nesse contexto, a pergunta pode ser respondida da seguinte maneira:

O que falta aos pregadores de hoje é o ensino bíblico consistente e sério, caracterizado por uma fundamentação sólida nas Escrituras Sagradas. É necessário um compromisso com a Palavra de Deus, uma dedicação à estudo da Bíblia e uma habilidade em aplicá-la às necessidades espirituais dos crentes.

Além disso, os pregadores podem falta a amizade real com as pessoas, a capacidade de compreender suas necessidades e doros, e a coragem de anunciar o Evangelho de forma clara e persuasiva. O que falta também é a perseverança em oração, como exemplificam textos da Bíblia (2 Tm 3.16-17), e a capacidade de crer na eficácia do ensino para mudar as pessoas.

Portanto, o que falta aos pregadores de hoje é um compromisso com Deus, a Palavra de Deus, e uma dedicação à estudo da Bíblia, juntamente com a coragem de anunciar o Evangelho e a perseverança em oração.

In [15]:
display(Markdown(final_rag_answer))

A questão é inspirada pelo livro "Somos Todos Teólogos" do reverendo R.C. Sproul, e mais especificamente pela página 240 da obra.

Nesse contexto, a pergunta pode ser respondida da seguinte maneira:

O que falta aos pregadores de hoje é o ensino bíblico consistente e sério, caracterizado por uma fundamentação sólida nas Escrituras Sagradas. É necessário um compromisso com a Palavra de Deus, uma dedicação à estudo da Bíblia e uma habilidade em aplicá-la às necessidades espirituais dos crentes.

Além disso, os pregadores podem falta a amizade real com as pessoas, a capacidade de compreender suas necessidades e doros, e a coragem de anunciar o Evangelho de forma clara e persuasiva. O que falta também é a perseverança em oração, como exemplificam textos da Bíblia (2 Tm 3.16-17), e a capacidade de crer na eficácia do ensino para mudar as pessoas.

Portanto, o que falta aos pregadores de hoje é um compromisso com Deus, a Palavra de Deus, e uma dedicação à estudo da Bíblia, juntamente com a coragem de anunciar o Evangelho e a perseverança em oração.

## Anwser

In [7]:
display(Markdown(f"**Busca:** {question}\n\n"))
display(Markdown(final_answer))

**Busca:** O que falta aos pregadores de hoje?



A partir do contexto fornecido, encontrei os seguintes trechos relacionados ao assunto "O que falta aos pregadores de hoje?" :

* Do documento com o título "Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf" (página 45):
"Pentecostes significa dor, mas o que ma is expe rimentamos é prazer; significa peso; mas nós amamos a comodidade. Pentecostes significa prisão, e, no entanto, a maioria dos crentes faria qualquer coisa, menos ir para a prisão por amor a Cristo."

* Do documento com o título "Conselhos para obreiros - Charles H Spurgeon.pdf" (página 11):
"Nós precisamos nos enver gonhar de não sentir vergonha; precisamos chorar por n ão termos lágrimas; precisamos nos humilhar por haver perdido a humildade de servo de Deus; gemer por não sentirmos peso pelos perdi dos; irar-nos contra n ós mesmos por não termos ódio do monopólio que o diabo exerce nestes dias do fim, e nos punir pelo fato de o mundo estar-se dando tão bem conosco, que nem precisa perseguir-nos."

Esses trechos sugerem que os pregadores de hoje em dia podem estar falta de características como humildade, dor, lágrimas e amor a Cristo, o que pode estar afetando sua eficácia na pregação.

## Debug

In [8]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Trecho:** pregadores, precisamos nos enver gonhar de não sentir vergonha; 
precisamos chorar por n ão termos lágrimas; precisamos nos humilhar 
por haver perdido a humildade de servo de Deus; gemer por não 
sentirmos peso pelos perdi dos; irar -nos contra n ós mesmos por não 
termos ódio do monopólio que o diabo exerce nestes dias do fim, e 
nos punir pelo fato de o mundo estar -se dando tão bem conosco, que 
nem precisa perseguir -nos. 
Pentecostes significa dor, mas o que ma is expe rimentamos  é 
prazer; significa peso; mas nós amamos a comodidade. Pentecostes 
significa prisão, e, no entanto, a maioria dos crentes faria qualquer 
coisa, menos ir para a prisão por amor a Cristo. Se revivêss emos a 
experiência do pentecostes, talvez muitos de nós fossem parar na

**Fonte:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 77}

**Trecho:** na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem 
tem coração quebrantado é capaz de levar outros ao 
quebrantamento. Ir mãos, nós temos equipamentos, mas não temos 
poder; temos ação, mas não unção; barulho,  mas não avivamento. 
Somos dogmát icos, mas  não dinâmicos!  
Todas as eras têm iniciado com fogo, e todas as vidas, sejam de

**Fonte:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 11}

**Trecho:** “Pensei que ele não havia se preocupado muito conosco, porque sabia que, por onde quer que ele
passe, argumenta com seus anfitriões, e não havia feito assim conosco. Cheguei a pensar que
havíamos irritado o sr. Whitefield, mas agora percebo como ele foi amável conosco, falando-nos
dessa maneira.”
Em seguida chamou suas filhas: “Subam, meninas. Vejam o que o homem de Deus escreveu na
vidraça: ‘Falta-lhes uma coisa!’. Chamem seu pai”. O coronel subiu ao aposento de hóspedes e
também leu a frase: “Falta-lhes uma coisa!”. Assim, ao redor da cama onde o homem de Deus
havia dormido, ajoelharam-se e pediram a Deus que lhes desse a coisa que faltava. E, ali mesmo,
antes  que deixassem o quarto, encontraram  o que lhes  faltava;  e toda aquela  família  pôde
regozijar-se em Jesus.
Não faz muito tempo, encontrei-me com um amigo em cuja igreja há uma irmã que preserva
aquele pedaço de vidro como herança de família. Se você não conseguir aconselhar ou admoestar

In [9]:
sources = db.similarity_search_with_score(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source[0].page_content}"))
    display(Markdown(f"**Trecho:** {source[0].metadata}"))
    display(Markdown(f"**Score:** {source[1]}"))

**Fonte:** pregadores, precisamos nos enver gonhar de não sentir vergonha; 
precisamos chorar por n ão termos lágrimas; precisamos nos humilhar 
por haver perdido a humildade de servo de Deus; gemer por não 
sentirmos peso pelos perdi dos; irar -nos contra n ós mesmos por não 
termos ódio do monopólio que o diabo exerce nestes dias do fim, e 
nos punir pelo fato de o mundo estar -se dando tão bem conosco, que 
nem precisa perseguir -nos. 
Pentecostes significa dor, mas o que ma is expe rimentamos  é 
prazer; significa peso; mas nós amamos a comodidade. Pentecostes 
significa prisão, e, no entanto, a maioria dos crentes faria qualquer 
coisa, menos ir para a prisão por amor a Cristo. Se revivêss emos a 
experiência do pentecostes, talvez muitos de nós fossem parar na

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 45}

**Score:** 398.0030822753906

**Fonte:** na sua fidelidade ao Pai  (prepa rou-se durante tanto tempo pa ra 
pregar por tão curto período); grande em sua submissão  ao Espírito 
(anda va ou parava de acordo com as orientações dele); grande nas 
afirm ações que fez  sobre o Filho (apon tando Jesus como “o Cordeiro 
de Deus, que tira o pecado do mundo ”, apesar de não te r-se avistado 
com ele antes).  
João era uma “Voz”. A maioria dos pregadores não passa de ecos, 
pois, se prestarmos bem atenção, saberemos dizer quais os livros 
que andaram lendo, e notaremos que citaram muito pouco do Livro. E 
hoje, só uma Voz, voz de um pr ofeta enviado do céu par a pregar aos 
pregadores , conseguiria despertar o co ração dos homens. S ó quem 
tem coração quebrantado é capaz de levar outros ao 
quebrantamento. Ir mãos, nós temos equipamentos, mas não temos 
poder; temos ação, mas não unção; barulho,  mas não avivamento. 
Somos dogmát icos, mas  não dinâmicos!  
Todas as eras têm iniciado com fogo, e todas as vidas, sejam de

**Trecho:** {'source': './books/import/Por que tarda o pleno Avivamento - Leonard Ravenhill.pdf', 'page': 77}

...

**Trecho:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 11}

**Score:** 410.0894775390625

**Fonte:** ela mesma. 
Se Jesus tivesse dito a ela antes de fazê-la: "Há apenas duas vagas disponíveis para você. Você pode ser uma negra ou então uma branca pobre. Qual dessas?", ela teria se retraído e se contorcido antes de finalmente responder: "Tudo bem, faça-me negra então, mas não uma dessas maloqueiras". E el e a teria feito uma mulher negra limpa e 
asseada, ela mesma numa versão negra. 
 Bem, um dia a sra. Turpin vai ao consultório médico e vê-se cercada pelo tipo de pessoas que ela despreza. De repente uma jovem com acne atra vessa a passos largos a sala de espera, golpeia 
a sra. Turpin com um livro e tenta estrangulá-l a. Quando a ordem é finalmente restaurada, a 
sra. Turpin, como se estivesse ag uardando uma revelação, pergunta: 
— O que você tem a me dizer? 
— Volte para o inferno de onde você ve io, sua porca velha! — grita a moça. 
A sra. Turpin desaba. O seu mundo fez-se em pedaços. O Deus que ela havia moldado à

**Trecho:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 18}

**Score:** 416.1697692871094